<a href="https://colab.research.google.com/github/giseletoledo/tradutor/blob/main/tradutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Tradução de PDF*

In [ ]:
Estou testando algumas bibliotecas para traduzir um PDF dos meus estudos de Espanhol. Pretendo testar com inglês também. Uso de pdf2image e googletrans

In [3]:
pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.1 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=ae710cdaa3ff6abafaef5112ae8786715046c717366cb0ec4a9623ca7f197db5
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:


In [2]:
pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [7]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (768 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [14]:
pip install --upgrade PyPDF2

In [13]:
from PyPDF2 import PdfReader
from googletrans import Translator
from ipywidgets import FileUpload
from IPython.display import display, HTML
import io
import re
import logging

# Configuração do logging para depuração
logging.basicConfig(filename='app.log', level=logging.DEBUG)

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens.

    Args:
        pdf_bytes: Conteúdo binário do PDF.

    Returns:
        str: Texto extraído ou None em caso de erro.
    """
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        logging.exception("Erro ao extrair texto do PDF")
        return None

def format_text(text):
    """Formata o texto extraído para identificar títulos e subtítulos.

    Args:
        text: O texto extraído do PDF.

    Returns:
        str: Texto formatado com detecção de títulos e subtítulos.
    """
    # Expressão regular para identificar títulos principais (1., 2., 3., etc.)
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    # Expressão regular para identificar subtítulos (1.1, 2.1, 2.2, etc.)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)

    # Substituindo os títulos principais para adicionar um cabeçalho em HTML
    formatted_text = re.sub(title_pattern, r'<h1>\g<0></h1>', text)

    # Substituindo os subtítulos para adicionar um subtítulo em HTML
    formatted_text = re.sub(subtitle_pattern, r'<h2>\g<0></h2>', formatted_text)

    return formatted_text

def handle_pdf_upload(change):
    """Processa o upload do PDF.

    Args:
        change: Evento de mudança do widget FileUpload.
    """
    try:
        if not change.new:
            display(HTML("<h3>Erro:</h3><pre>Nenhum arquivo foi carregado.</pre>"))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("<h3>Erro:</h3><pre>O conteúdo do arquivo está vazio ou não foi carregado corretamente.</pre>"))
            return

        text = extract_text_from_pdf(content)

        if not text:
            display(HTML("<h3>Erro:</h3><pre>Não foi encontrado texto no PDF ou houve um problema na extração.</pre>"))
            return

        # Verificar se o texto extraído não está vazio
        if len(text.strip()) == 0:
            display(HTML("<h3>Erro:</h3><pre>O texto extraído está vazio.</pre>"))
            return

        # Formatar o texto para identificar títulos e subtítulos
        formatted_text = format_text(text)

        # Traduzir o texto formatado
        translator = Translator()
        text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]
        translated_text = ""

        for part in text_parts:
            translated_text += translator.translate(part, dest='pt').text

        # Exibir a tradução formatada
        display(HTML(f"<h3>Texto traduzido:</h3><pre>{translated_text}</pre>"))
    except Exception as e:
        logging.exception("Erro ao processar o arquivo")
        display(HTML(f"<h3>Erro ao processar o arquivo:</h3><pre>{str(e)}</pre>"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)


FileUpload(value={}, accept='.pdf', description='Upload')

In [14]:
from PyPDF2 import PdfReader
from googletrans import Translator
from ipywidgets import FileUpload
from IPython.display import display, HTML
import io
import re
import logging

# Configuração do logging para depuração
logging.basicConfig(filename='app.log', level=logging.DEBUG)

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens.

    Args:
        pdf_bytes: Conteúdo binário do PDF.

    Returns:
        str: Texto extraído ou None em caso de erro.
    """
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        logging.exception("Erro ao extrair texto do PDF")
        return None

def format_text(text):
    """Formata o texto extraído para identificar títulos e subtítulos.

    Args:
        text: O texto extraído do PDF.

    Returns:
        str: Texto formatado com detecção de títulos e subtítulos.
    """
    # Expressão regular para identificar títulos principais (1., 2., 3., etc.)
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    # Expressão regular para identificar subtítulos (1.1, 2.1, 2.2, etc.)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)

    # Substituindo os títulos principais para adicionar um cabeçalho em HTML
    formatted_text = re.sub(title_pattern, r'<h1>\g<0></h1>', text)

    # Substituindo os subtítulos para adicionar um subtítulo em HTML
    formatted_text = re.sub(subtitle_pattern, r'<h2>\g<0></h2>', formatted_text)

    return formatted_text

def handle_pdf_upload(change):
    """Processa o upload do PDF.

    Args:
        change: Evento de mudança do widget FileUpload.
    """
    try:
        if not change.new:
            display(HTML("<h3>Erro:</h3><pre>Nenhum arquivo foi carregado.</pre>"))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("<h3>Erro:</h3><pre>O conteúdo do arquivo está vazio ou não foi carregado corretamente.</pre>"))
            return

        text = extract_text_from_pdf(content)

        if not text:
            display(HTML("<h3>Erro:</h3><pre>Não foi encontrado texto no PDF ou houve um problema na extração.</pre>"))
            return

        # Verificar se o texto extraído não está vazio
        if len(text.strip()) == 0:
            display(HTML("<h3>Erro:</h3><pre>O texto extraído está vazio.</pre>"))
            return

        # Formatar o texto para identificar títulos e subtítulos
        formatted_text = format_text(text)

        # Traduzir o texto formatado
        translator = Translator()
        text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]
        translated_text = ""

        for part in text_parts:
            translated_text += translator.translate(part, dest='pt').text

        # Exibir o texto original e a tradução, com fundo cinza para o original
        display(HTML(f"""
            <h3>Texto original:</h3>
            <div style="background-color: #f0f0f0; padding: 10px;">
                <pre>{formatted_text}</pre>
            </div>
            <h3>Texto traduzido:</h3>
            <pre>{translated_text}</pre>
        """))
    except Exception as e:
        logging.exception("Erro ao processar o arquivo")
        display(HTML(f"<h3>Erro ao processar o arquivo:</h3><pre>{str(e)}</pre>"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)


FileUpload(value={}, accept='.pdf', description='Upload')

In [7]:
from PyPDF2 import PdfReader
from googletrans import Translator
from ipywidgets import FileUpload
from IPython.display import display, HTML
import io
import re
import logging

# Configuração do logging para depuração
logging.basicConfig(filename='app.log', level=logging.DEBUG)

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens.

    Args:
        pdf_bytes: Conteúdo binário do PDF.

    Returns:
        str: Texto extraído ou None em caso de erro.
    """
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        logging.exception("Erro ao extrair texto do PDF")
        return None

def format_text(text):
    """Formata o texto extraído para identificar títulos e subtítulos.

    Args:
        text: O texto extraído do PDF.

    Returns:
        str: Texto formatado com detecção de títulos e subtítulos.
    """
    # Expressão regular para identificar títulos principais (1., 2., 3., etc.)
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    # Expressão regular para identificar subtítulos (1.1, 2.1, 2.2, etc.)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)

    # Substituindo os títulos principais para adicionar um cabeçalho em HTML
    formatted_text = re.sub(title_pattern, r'<h1>\g<0></h1>', text)

    # Substituindo os subtítulos para adicionar um subtítulo em HTML
    formatted_text = re.sub(subtitle_pattern, r'<h2>\g<0></h2>', formatted_text)

    return formatted_text

def clean_spacing(text):
    """Remove espaçamentos estranhos e normaliza o texto.

    Args:
        text: O texto a ser limpo.

    Returns:
        str: Texto sem espaçamentos desnecessários.
    """
    # Remover espaçamentos múltiplos entre as palavras, mas preservar as quebras de linha
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('. ', '.\n')  # Manter quebras de parágrafo
    return text.strip()

def handle_pdf_upload(change):
    """Processa o upload do PDF.

    Args:
        change: Evento de mudança do widget FileUpload.
    """
    try:
        if not change.new:
            display(HTML("<h3>Erro:</h3><pre>Nenhum arquivo foi carregado.</pre>"))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("<h3>Erro:</h3><pre>O conteúdo do arquivo está vazio ou não foi carregado corretamente.</pre>"))
            return

        text = extract_text_from_pdf(content)

        if not text:
            display(HTML("<h3>Erro:</h3><pre>Não foi encontrado texto no PDF ou houve um problema na extração.</pre>"))
            return

        # Verificar se o texto extraído não está vazio
        if len(text.strip()) == 0:
            display(HTML("<h3>Erro:</h3><pre>O texto extraído está vazio.</pre>"))
            return

        # Limpar espaçamentos estranhos e formatar o texto
        clean_text = clean_spacing(text)
        formatted_text = format_text(clean_text)

        # Traduzir o texto formatado
        translator = Translator()
        text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]

        translated_pairs = []  # Para armazenar pares de original e traduzido

        for part in text_parts:
            original_part = part
            translated_part = translator.translate(part, dest='pt').text
            translated_pairs.append((original_part, translated_part))

        # Construir o HTML para exibir o original e a tradução intercalados
        result_html = "<h3>Texto original e tradução:</h3><ol>"

        for original, translated in translated_pairs:
            # Dividindo por blocos de aproximadamente 8 linhas
            original_lines = original.split("\n")
            translated_lines = translated.split("\n")

            # Agrupar em blocos de 8 linhas
            block_size = 8
            for i in range(0, max(len(original_lines), len(translated_lines)), block_size):
                original_block = "\n".join(original_lines[i:i + block_size])
                translated_block = "\n".join(translated_lines[i:i + block_size])

                # Adiciona o bloco de texto original e traduzido na lista com estilo justificado
                result_html += f"""
                    <li>
                        <strong>Original:</strong>
                        <pre style='background-color: #f0f0f0; text-align: left; white-space: pre-wrap;'>{original_block}</pre>
                        <strong>Traduzido:</strong>
                        <pre style='text-align: left; white-space: pre-wrap;'>{translated_block}</pre>
                    </li>
                """

        result_html += "</ol>"

        # Exibir o resultado formatado
        display(HTML(result_html))

    except Exception as e:
        logging.exception("Erro ao processar o arquivo")
        display(HTML(f"<h3>Erro ao processar o arquivo:</h3><pre>{str(e)}</pre>"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)


FileUpload(value={}, accept='.pdf', description='Upload')